# Runtime and peak memory summary

Measure runtime and peak memory for small circuits across QuASAr and baseline simulators.

In [ ]:
import time, tracemalloc
import pandas as pd
from benchmarks.circuits import ghz_circuit, clifford_ec_circuit
from quasar.circuit import Circuit
from quasar.simulation_engine import SimulationEngine
from quasar.backends import AerStatevectorBackend, StimBackend
from quasar.analyzer import CircuitAnalyzer


def run_quasar(circuit: Circuit) -> tuple[float, float]:
    engine = SimulationEngine()
    analyzer = CircuitAnalyzer(circuit, estimator=engine.planner.estimator)
    analysis = analyzer.analyze()
    plan = engine.planner.plan(circuit, analysis=analysis)
    _, metrics = engine.scheduler.run(circuit, plan, analysis=analysis, instrument=True)
    return metrics.cost.time, metrics.cost.memory


def run_statevector(circuit: Circuit) -> tuple[float, float]:
    backend = AerStatevectorBackend()
    backend.load(circuit.num_qubits)
    for gate in circuit.gates:
        backend.apply_gate(gate.gate, gate.qubits, gate.params)
    tracemalloc.start(); tracemalloc.reset_peak()
    start = time.perf_counter()
    backend.statevector()
    runtime = time.perf_counter() - start
    _, peak = tracemalloc.get_traced_memory(); tracemalloc.stop()
    return runtime, peak


def run_stim(circuit: Circuit) -> tuple[float, float]:
    backend = StimBackend()
    backend.load(circuit.num_qubits)
    for gate in circuit.gates:
        backend.apply_gate(gate.gate, gate.qubits, gate.params)
    tracemalloc.start(); tracemalloc.reset_peak()
    start = time.perf_counter()
    backend.extract_ssd()
    runtime = time.perf_counter() - start
    _, peak = tracemalloc.get_traced_memory(); tracemalloc.stop()
    return runtime, peak


circuits = {
    'ghz3': ghz_circuit(3, use_classical_simplification=False),
    'clifford_ec': clifford_ec_circuit(),
}

rows = []
for name, circuit in circuits.items():
    for sim_name, runner in [('QuASAr', run_quasar), ('Statevector', run_statevector), ('Stim', run_stim)]:
        runtime, peak = runner(circuit)
        rows.append({'circuit': name, 'simulator': sim_name, 'runtime_s': runtime, 'peak_memory_bytes': peak})

df = pd.DataFrame(rows)
df